# Perform phenotypic volume calculation <a class="tocSkip">

For major cell type populations (T cells, myeloid, tumor cells labelled with Echidna clones), perform Phenotypic Volume calculation and compare across clinical covariates

Author: Kevin Hoffer-Hawlik

## Setup

In [1]:
import warnings
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
import matplotlib.pyplot as plt
import scvi
import seaborn as sns
import scipy.stats
import pickle
import seaborn as sns
import decipher as dc

/home/kh3205/anaconda3/envs/decipher/lib/python3.11/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/kh3205/anaconda3/envs/decipher/lib/python3.11/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/kh3205/anaconda3/envs/decipher/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
warnings.simplefilter(action='ignore',)

In [3]:
warnings.simplefilter(action='ignore',)

## Load data and set global settings

In [4]:
clinical_outcomes = pd.read_csv("clin_022625.csv")
clinical_outcomes.set_index(keys = 'sample', inplace=True)

In [5]:
tcells_decipher = sc.read_h5ad('tcells_decipher_041024.h5ad')
tcells_decipher.shape

(20512, 8170)

In [ ]:
myeloid = sc.read_h5ad('myeloid_scvi_annotated_032025.h5ad')
myeloid.shape

(18152, 2041)

In [ ]:
ct_ad = sc.read_h5ad('contact_tracing_merged_022625.h5ad')
ct_ad.shape

In [8]:
# harmonize clinical metadata
for adata in [myeloid, tcells_decipher]:
    adata.obs['time'] = adata.obs['sample'].replace(clinical_outcomes['time'])
    adata.obs['treated'] = adata.obs['sample'].replace(clinical_outcomes['treated'])
    adata.obs['responder'] = adata.obs['sample'].replace(clinical_outcomes['responder'])
    adata.obs['treat_resp'] = adata.obs['sample'].replace(clinical_outcomes['treat_resp'])

In [9]:
# rename and order categories as follows
# set color dictionaries
clinical_variable_palette = {
    'Expanded': 'firebrick',
    'Non-expanded': 'salmon',
    'TCR Not Available': 'lightgray',
    'Pre ICI': '#7f7f7f', # grey
    'On ICI': '#1f77b4', # blue
    'Post ICI': '#2ca02c', # green
    'Responder': 'midnightblue',
    'Non-responder': 'lightsteelblue',
    'LN': 'red',
    'Liver': 'darkorange',
    'Mucosal (oral)': 'green',
    'Mucosal (rectal)': 'darkgreen',
    'Skin':'steelblue',
    'Untreated': '#7f7f7f', # grey
    'Treated': '#1f77b4', # blue
}
clinical_variable_cmaps = {
    'time_to_ICI': 'plasma', # 'rocket'
    'OS': 'plasma', # 'rocket'
}

immune_cell_type_palette = {
    'CD4+ T(cm)': 'navajowhite',
    'CD4+ T': 'darkorange',
    'Treg': 'red', # trajectory color
    'CD8+ T(cm)': 'lightskyblue',
    'CD8+ T': 'blue',
    'NK_1': 'lightgreen',
    'NK_2': 'darkgreen',
    'Cycling Immune Cell': 'indigo',
    'CD4': 'darkorange', # trajectory color
    'CD8': 'blue' # trajectory color
}

myeloid_cell_type_palette = {
    'Monocytes': 'mediumpurple',
    'cDC1': 'lightgreen',
    'cDC2': 'lightseagreen',
    'DC3': 'darkgreen',
    'Macrophages (antigen-pres.)': 'navajowhite',
    'Macrophages (phagocytic)': 'darkorange',
    'Macrophages (M1-like)': 'red',
    'Macrophages (M2-like)': 'blue',
    'Macrophages (cycling)': 'indigo',
    'Monocyte-M1':'red', # trajectory color
    'Monocyte-M2':'blue', # trajectory color
}
myeloid_cell_type_decipher_palette = {
    'Monocytes': 'tab:blue',
    'Macrophages (antigen-pres.)': 'tab:orange',
    'Macrophages (phagocytic)': 'tab:green',
    'Macrophages (M1-like)': 'tab:red',
    'Macrophages (M2-like)': 'tab:purple',
}

# perform PV

In [10]:
import importlib
import PV
from PV import *
importlib.reload(PV)

<module 'PV' from '/home/kh3205/melanoma-project/PV.py'>

## tcells

* cd8_cd4_treg x pre_on_post
* r_nr x pre_on_post
* cd8_cd4_treg x r_nr

In [ ]:
keep = list(tcells_decipher.var[tcells_decipher.var['highly_variable_rank']<2000].index)

In [26]:
tcells_decipher.obs['cell_type_khh'] = tcells_decipher.obs['cell_type_khh'].replace({
    'Naive CD8+ T': 'CD8+ T',
    'Naive CD4+ T': 'CD4+ T',
})
tcells_decipher.obs['cell_type_khh'].unique()

['CD4+ T', 'Treg', 'CD8+ T']
Categories (3, object): ['CD4+ T', 'CD8+ T', 'Treg']

In [ ]:
# see hvg_rank_plots notebook
tcells_decipher = tcells_decipher[:,keep]
tcells_decipher.shape

(20512, 2151)

In [ ]:
# cd8_cd4_treg x pre_on_post
tcells_decipher.obs_names = tcells_decipher.obs['cell_type_khh'].astype('str') + "_" + tcells_decipher.obs['time'].astype('str')
PV = phenotypic_volume(tcells_decipher, layer='log_counts')
PV.to_csv('PV_022725/tcell_ct_time.csv')

96
CD4+ T_on
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [ ]:
# r_nr x pre_on_post
tcells_decipher.obs_names = tcells_decipher.obs['responder'].astype('str') + "_" + tcells_decipher.obs['time'].astype('str')
PV = phenotypic_volume(tcells_decipher, layer='log_counts')
PV.to_csv('PV_022725/tcell_responder_time.csv')

In [ ]:
# cd8_cd4_treg x r_nr
tcells_decipher.obs_names = tcells_decipher.obs['cell_type_khh'].astype('str') + "_" + tcells_decipher.obs['responder'].astype('str')
PV = phenotypic_volume(tcells_decipher, layer='log_counts')
PV.to_csv('PV_022725/tcell_ct_responder.csv')

## myeloid

* tx_untx x r_nr
* r_nr x pre_on_post
* cell_type_main x r_nr
* cell_type_main x pre_on_post
* cell_type_main x tx_untx

In [12]:
# filter out rare / noisy subtypes
myeloid = myeloid[myeloid.obs['cell_type_fine'].isin(
    ['Antigen-presenting Macrophage','Phagocytic Macrophage',
     'M1-like Macrophage','M2-like Macrophage',
     'cDC1','cDC2','DC3',
     'Monocyte',]
)]

In [13]:
# myeloid.layers["counts"] = myeloid.X.copy() # save raw counts
# sc.pp.normalize_total(myeloid)
# sc.pp.log1p(myeloid)
# myeloid.layers["normalized_counts"] = myeloid.X.copy()
# myeloid = myeloid[:,myeloid_decipher.var_names]
# print(myeloid.shape)

In [14]:
myeloid.obs['cell_type_fine'].value_counts()

cell_type_fine
Phagocytic Macrophage            6888
M2-like Macrophage               3149
Antigen-presenting Macrophage    2382
M1-like Macrophage               1540
Monocyte                         1421
cDC2                             1070
cDC1                              484
DC3                               387
Name: count, dtype: int64

In [15]:
myeloid.obs['cell_type_main'] = myeloid.obs['cell_type_fine'].replace({
    'Antigen-presenting Macrophage':'Macrophage',
    'Phagocytic Macrophage':'Macrophage',
    'M1-like Macrophage':'Macrophage',
    'M2-like Macrophage':'Macrophage',
    'DC3':'DC',
    'cDC2':'DC',
    'cDC1':'DC',
    'Monocyte':'Monocyte'
})

In [16]:
myeloid.obs['cell_type_main'].value_counts()

cell_type_main
Macrophage    13959
DC             1941
Monocyte       1421
Name: count, dtype: int64

In [17]:
# tx_untx x r_nr
myeloid.obs_names = myeloid.obs['treated'].astype('str') + "_" + myeloid.obs['responder'].astype('str')
PV = phenotypic_volume(myeloid, layer='normalized_counts')
PV.to_csv('PV_022725/myeloid_treated_responder.csv')

88
Treated_Non-responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
Untreated_Non-responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
Untreated_Responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
Treated_Responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53

In [18]:
# r_nr x pre_on_post
myeloid.obs_names = myeloid.obs['responder'].astype('str') + "_" + myeloid.obs['time'].astype('str')
PV = phenotypic_volume(myeloid, layer='normalized_counts')
PV.to_csv('PV_022725/myeloid_responder_time.csv')

87
Non-responder_on
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
Non-responder_pre
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
Responder_pre
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
Non-responder_post
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [19]:
# cell_type_main x r_nr
myeloid.obs_names = myeloid.obs['cell_type_main'].astype('str') + "_" + myeloid.obs['responder'].astype('str')
PV = phenotypic_volume(myeloid, layer='normalized_counts')
PV.to_csv('PV_022725/myeloid_ct_responder.csv')

201
Macrophage_Non-responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
DC_Non-responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [20]:
# cell_type_main x pre_on_post
myeloid.obs_names = myeloid.obs['cell_type_main'].astype('str') + "_" + myeloid.obs['time'].astype('str')
PV = phenotypic_volume(myeloid, layer='normalized_counts')
PV.to_csv('PV_022725/myeloid_ct_time.csv')

203
Macrophage_on
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
DC_on
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94

In [21]:
# cell_type_main x tx_untx
myeloid.obs_names = myeloid.obs['cell_type_main'].astype('str') + "_" + myeloid.obs['treated'].astype('str')
PV = phenotypic_volume(myeloid, layer='normalized_counts')
PV.to_csv('PV_022725/myeloid_ct_treated.csv')

360
Macrophage_Treated
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271

In [22]:
macrophages = myeloid[myeloid.obs['cell_type_main'].isin(['Macrophage','Monocyte'])]

In [23]:
# cell_type_fine x r_nr
macrophages.obs_names = macrophages.obs['cell_type_fine'].astype('str') + "_" + macrophages.obs['responder'].astype('str')
PV = phenotypic_volume(macrophages, layer='normalized_counts')
PV.to_csv('PV_022725/macrophages_ct_responder.csv')

114
Antigen-presenting Macrophage_Non-responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
Phagocytic Macrophage_Non-responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
M2-like Macrophage_Non-responder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


In [24]:
# cell_type_fine x pre_on_post
macrophages.obs_names = macrophages.obs['cell_type_fine'].astype('str') + "_" + macrophages.obs['time'].astype('str')
PV = phenotypic_volume(macrophages, layer='normalized_counts')
PV.to_csv('PV_022725/macrophages_ct_time.csv')

109
Antigen-presenting Macrophage_on
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
Phagocytic Macrophage_on
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
M2-like Macrophage_on
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
9

In [25]:
# cell_type_fine x tx_untx
macrophages.obs_names = macrophages.obs['cell_type_fine'].astype('str') + "_" + macrophages.obs['treated'].astype('str')
PV = phenotypic_volume(macrophages, layer='normalized_counts')
PV.to_csv('PV_022725/macrophages_ct_treated.csv')

177
Antigen-presenting Macrophage_Treated
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
Phagocytic Macrophage_Treated
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


## tumor clones

* pt x growing_shrinking_stable x tx_untx
* pt x growing_shrinking_stable x pre_on_post

In [14]:
for adata in [myeloid, tcells_decipher]:
    del adata

In [15]:
tumors = ct_ad[ct_ad.obs['cell_type_ct_dpt'].isin([
    'growing_Tumor_clones','shrinking_Tumor_clones','stable_Tumor_clones'
])]
del ct_ad
tumors.shape

(84229, 18211)

In [ ]:
# limit to 5K HVGs across patients
sc.pp.highly_variable_genes(tumors, flavor='seurat', batch_key = 'patient', n_top_genes = 5000, subset=True, inplace=True)
tumors.shape

(84229, 5000)

In [20]:
tumors.layers['normalized_counts'] = tumors.X.copy()

In [23]:
# pt x growing_shrinking_stable x tx_untx
tumors.obs_names = tumors.obs['patient'].astype('str') + "_" + tumors.obs['cell_type_ct_dpt'].astype('str') + "_" + tumors.obs['treated'].astype('str')
PV = phenotypic_volume(tumors, layer='normalized_counts')
PV.to_csv('PV_022725/tumor_pt_clone_treated.csv')

51185
F01_shrinking_Tumor_clones_Treated
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
2

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



25744
25745
25746
25747
25748
25749
25750
25751
25752
25753
25754
25755
25756
25757
25758
25759
25760
25761
25762
25763
25764
25765
25766
25767
25768
25769
25770
25771
25772
25773
25774
25775
25776
25777
25778
25779
25780
25781
25782
25783
25784
25785
25786
25787
25788
25789
25790
25791
25792
25793
25794
25795
25796
25797
25798
25799
25800
25801
25802
25803
25804
25805
25806
25807
25808
25809
25810
25811
25812
25813
25814
25815
25816
25817
25818
25819
25820
25821
25822
25823
25824
25825
25826
25827
25828
25829
25830
25831
25832
25833
25834
25835
25836
25837
25838
25839
25840
25841
25842
25843
25844
25845
25846
25847
25848
25849
25850
25851
25852
25853
25854
25855
25856
25857
25858
25859
25860
25861
25862
25863
25864
25865
25866
25867
25868
25869
25870
25871
25872
25873
25874
25875
25876
25877
25878
25879
25880
25881
25882
25883
25884
25885
25886
25887
25888
25889
25890
25891
25892
25893
25894
25895
25896
25897
25898
25899
25900
25901
25902
25903
25904
25905
25906
25907
25908
25909
2591

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



30460
30461
30462
30463
30464
30465
30466
30467
30468
30469
30470
30471
30472
30473
30474
30475
30476
30477
30478
30479
30480
30481
30482
30483
30484
30485
30486
30487
30488
30489
30490
30491
30492
30493
30494
30495
30496
30497
30498
30499
30500
30501
30502
30503
30504
30505
30506
30507
30508
30509
30510
30511
30512
30513
30514
30515
30516
30517
30518
30519
30520
30521
30522
30523
30524
30525
30526
30527
30528
30529
30530
30531
30532
30533
30534
30535
30536
30537
30538
30539
30540
30541
30542
30543
30544
30545
30546
30547
30548
30549
30550
30551
30552
30553
30554
30555
30556
30557
30558
30559
30560
30561
30562
30563
30564
30565
30566
30567
30568
30569
30570
30571
30572
30573
30574
30575
30576
30577
30578
30579
30580
30581
30582
30583
30584
30585
30586
30587
30588
30589
30590
30591
30592
30593
30594
30595
30596
30597
30598
30599
30600
30601
30602
30603
30604
30605
30606
30607
30608
30609
30610
30611
30612
30613
30614
30615
30616
30617
30618
30619
30620
30621
30622
30623
30624
30625
3062

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



49740
49741
49742
49743
49744
49745
49746
49747
49748
49749
49750
49751
49752
49753
49754
49755
49756
49757
49758
49759
49760
49761
49762
49763
49764
49765
49766
49767
49768
49769
49770
49771
49772
49773
49774
49775
49776
49777
49778
49779
49780
49781
49782
49783
49784
49785
49786
49787
49788
49789
49790
49791
49792
49793
49794
49795
49796
49797
49798
49799
49800
49801
49802
49803
49804
49805
49806
49807
49808
49809
49810
49811
49812
49813
49814
49815
49816
49817
49818
49819
49820
49821
49822
49823
49824
49825
49826
49827
49828
49829
49830
49831
49832
49833
49834
49835
49836
49837
49838
49839
49840
49841
49842
49843
49844
49845
49846
49847
49848
49849
49850
49851
49852
49853
49854
49855
49856
49857
49858
49859
49860
49861
49862
49863
49864
49865
49866
49867
49868
49869
49870
49871
49872
49873
49874
49875
49876
49877
49878
49879
49880
49881
49882
49883
49884
49885
49886
49887
49888
49889
49890
49891
49892
49893
49894
49895
49896
49897
49898
49899
49900
49901
49902
49903
49904
49905
4990

KeyboardInterrupt: 

In [ ]:
# pt x growing_shrinking_stable x pre_on_post
tumors.obs_names = tumors.obs['patient'].astype('str') + "_" + tumors.obs['cell_type_ct_dpt'].astype('str') + "_" + tumors.obs['time'].astype('str')
PV = phenotypic_volume(tumors, layer='normalized_counts')
PV.to_csv('PV_022725/tumor_pt_clone_time.csv')